In [1]:
# !pip install catboost -q
# !pip install optuna -q

import pandas as pd
import numpy as np
import sklearn
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, KBinsDiscretizer
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import category_encoders
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mlxtend.plotting import plot_decision_regions
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
import seaborn as sns
import copy
import time
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import silhouette_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score
import ast
import warnings

sklearn.set_config(transform_output="pandas")
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('../../data/nlp/nlp_tickets_train.csv', sep=';')
dt = pd.read_csv('../../data/nlp/nlp_tickets_test.csv', sep=';')
X_train, y_train = df["vector"], df["cluster"]
X_test, y_test = dt["vector"], dt["cluster"]

In [3]:
def vector_transform(data) -> np.ndarray:
    """
    Transform the input data into a list of numpy arrays,
    where each element is a vector representation of a text document.
    :param data: pd.Series
    :return: np.ndarray: Transformed data as a list of numpy arrays,
    where each element is a vector representation of a text document.
    """
    data = data.apply(ast.literal_eval)
    return np.vstack(data)

In [4]:
X_train = vector_transform(X_train)
X_test = vector_transform(X_test)

### Logistic regression

In [60]:
model = OneVsRestClassifier(LogisticRegression(solver='lbfgs'))
model.fit(X_train, y_train)
y_pred_proba = model.predict_proba(X_test)
y_pred = model.predict(X_test)
roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class="ovr")
print(f'Logistic regression roc_auc {roc_auc}')

Logistic regression roc_auc 0.9081537041286649


In [70]:
print("\nОтчет о классификации:\n", classification_report(y_test, y_pred))


Отчет о классификации:
               precision    recall  f1-score   support

           1       0.92      0.97      0.95      3066
           2       0.00      0.00      0.00       133
           3       0.80      0.01      0.03       310
           4       0.52      0.77      0.62      1473
           5       0.69      0.09      0.16       362
           6       0.36      0.59      0.44       524
           7       0.00      0.00      0.00       255
           8       0.00      0.00      0.00       251
           9       0.70      0.86      0.77       897
          10       0.70      0.28      0.40       283

    accuracy                           0.70      7554
   macro avg       0.47      0.36      0.34      7554
weighted avg       0.68      0.70      0.65      7554



### SVC

In [75]:
class Config:
    def __init__(self):
        self.C = 1.0 
        self.kernel = "rbf"
        self.class_weight = "balanced"

config = Config()

# Обучение модели
model = SVC(
    C=config.C,
    kernel=config.kernel,
    class_weight=config.class_weight,
    decision_function_shape="ovr",
    probability=True,
)
model.fit(X_train, y_train)
y_pred_proba = model.predict_proba(X_test)
y_pred = model.predict(X_test)

roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class="ovr")
print(f'SVC roc_auc {roc_auc}')

SVC roc_auc 0.9350905553202118


In [77]:
print("\nОтчет о классификации:\n", classification_report(y_test, y_pred))


Отчет о классификации:
               precision    recall  f1-score   support

           1       0.97      0.97      0.97      3066
           2       0.25      0.77      0.38       133
           3       0.53      0.60      0.56       310
           4       0.79      0.46      0.58      1473
           5       0.44      0.61      0.51       362
           6       0.65      0.90      0.75       524
           7       0.40      0.77      0.53       255
           8       0.25      0.32      0.28       251
           9       0.95      0.72      0.81       897
          10       0.74      0.44      0.55       283

    accuracy                           0.75      7554
   macro avg       0.60      0.65      0.59      7554
weighted avg       0.80      0.75      0.76      7554



### Catboost

In [9]:
class Config:
    def __init__(self):
        self.iterations = 1000
        self.learning_rate = 0.1 
        self.depth = 6 
        self.l2_leaf_reg = 3 
        self.loss_function = "MultiClass"
        self.eval_metric = "MultiClass"
        self.verbose = 100
        self.random_seed = 42

config = Config()

# Обучение модели
model = CatBoostClassifier(
    iterations=config.iterations,
    learning_rate=config.learning_rate,
    depth=config.depth,
    l2_leaf_reg=config.l2_leaf_reg,
    loss_function=config.loss_function,
    eval_metric=config.eval_metric,
    verbose=config.verbose,
    random_seed=config.random_seed,
)
model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)
y_pred = np.argmax(y_pred_proba, axis=1)
y_pred = y_pred + 1
roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class="ovr")

print(f'CatBoost roc_auc {roc_auc}')

0:	learn: 1.8545316	total: 212ms	remaining: 3m 32s
100:	learn: 0.3423164	total: 8.39s	remaining: 1m 14s
200:	learn: 0.2215941	total: 16.6s	remaining: 1m 5s
300:	learn: 0.1592655	total: 24.8s	remaining: 57.7s
400:	learn: 0.1186335	total: 33.9s	remaining: 50.6s
500:	learn: 0.0949126	total: 42.6s	remaining: 42.4s
600:	learn: 0.0759704	total: 51.2s	remaining: 34s
700:	learn: 0.0635899	total: 59.2s	remaining: 25.3s
800:	learn: 0.0553164	total: 1m 7s	remaining: 16.7s
900:	learn: 0.0492554	total: 1m 15s	remaining: 8.28s
999:	learn: 0.0439376	total: 1m 23s	remaining: 0us
CatBoost roc_auc 0.9835091518154669


In [137]:
print("\nОтчет о классификации:\n", classification_report(y_test, y_pred))


Отчет о классификации:
               precision    recall  f1-score   support

           1       0.98      0.98      0.98      3066
           2       0.72      0.59      0.65       133
           3       0.83      0.69      0.75       310
           4       0.78      0.88      0.82      1473
           5       0.76      0.75      0.75       362
           6       0.94      0.92      0.93       524
           7       0.85      0.85      0.85       255
           8       0.83      0.77      0.80       251
           9       0.90      0.89      0.89       897
          10       0.59      0.45      0.51       283

    accuracy                           0.88      7554
   macro avg       0.82      0.78      0.79      7554
weighted avg       0.88      0.88      0.88      7554



### Optuna + CatBoost

In [13]:
import numpy as np
import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import train_test_split

class Config:
    def __init__(self):
        self.verbose = 0
        self.random_seed = 42

config = Config()

def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 500, 2000),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.3),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1, 10),
        "loss_function": "MultiClass",
        "eval_metric": "MultiClass",
        "verbose": config.verbose,
        "random_seed": config.random_seed,
    }
    
    model = CatBoostClassifier(**params)
    model.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=50, verbose=False)
    
    y_pred_proba = model.predict_proba(X_valid)
    return roc_auc_score(y_valid, y_pred_proba, multi_class="ovr")

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=config.random_seed)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)
best_params = study.best_params
print('best_params', best_params)

[I 2025-03-13 17:12:59,023] A new study created in memory with name: no-name-94b5666f-328f-4f30-8fd2-ee03546feb64
[I 2025-03-13 17:14:46,991] Trial 0 finished with value: 0.9737443920986146 and parameters: {'iterations': 1256, 'learning_rate': 0.04641003910597425, 'depth': 6, 'l2_leaf_reg': 7.033466105378084}. Best is trial 0 with value: 0.9737443920986146.
[I 2025-03-13 17:14:54,684] Trial 1 finished with value: 0.9720670592856127 and parameters: {'iterations': 1255, 'learning_rate': 0.11003676029958401, 'depth': 4, 'l2_leaf_reg': 1.5558775129112727}. Best is trial 0 with value: 0.9737443920986146.
[I 2025-03-13 17:15:45,284] Trial 2 finished with value: 0.971423731198011 and parameters: {'iterations': 1502, 'learning_rate': 0.013716958648742216, 'depth': 5, 'l2_leaf_reg': 1.1459348912153393}. Best is trial 0 with value: 0.9737443920986146.
[I 2025-03-13 17:19:21,046] Trial 3 finished with value: 0.9725469386047297 and parameters: {'iterations': 1200, 'learning_rate': 0.01378945995649

best_params {'iterations': 510, 'learning_rate': 0.09211244667259973, 'depth': 8, 'l2_leaf_reg': 4.857966274106451}


In [15]:
model = CatBoostClassifier(**best_params, loss_function="MultiClass", eval_metric="MultiClass", verbose=config.verbose, random_seed=config.random_seed)
model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)
y_pred = np.argmax(y_pred_proba, axis=1)
y_pred = y_pred + 1

roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class="ovr")

In [17]:
roc_auc

0.9820860126791173

In [19]:
print("\nОтчет о классификации:\n", classification_report(y_test, y_pred))


Отчет о классификации:
               precision    recall  f1-score   support

           1       0.98      0.98      0.98      3066
           2       0.81      0.58      0.68       133
           3       0.83      0.65      0.73       310
           4       0.76      0.88      0.81      1473
           5       0.72      0.74      0.73       362
           6       0.92      0.92      0.92       524
           7       0.85      0.82      0.83       255
           8       0.85      0.77      0.81       251
           9       0.91      0.88      0.90       897
          10       0.62      0.45      0.52       283

    accuracy                           0.88      7554
   macro avg       0.83      0.77      0.79      7554
weighted avg       0.88      0.88      0.88      7554



- Классы с высокими показателями (f1-score > 0.80)\
Класс 1: (f1-score 0.98) – очень высокая точность и полнота, модель отлично классифицирует этот класс.\
Класс 6: (f1-score 0.92) – сбалансированная и высокая точность/полнота.\
Класс 7: (f1-score 0.83) – хороший баланс между precision и recall.\
Класс 9: (f1-score 0.90) – высокая точность и полнота.\
Класс 4: (f1-score 0.81) – хороший результат.\
Вывод: Для этих классов модель работает почти идеально, можно сказать, что она хорошо распознает их.

- Классы со средними результатами (f1-score 0.70 - 0.79)\
Класс 3: (f1-score 0.73) – precision 0.83, но recall 0.65, что указывает на наличие ложноотрицательных.\
Класс 5: (f1-score 0.73) – precision 0.72 и recall 0.74, сбалансированное предсказание.\
Класс 8: (f1-score 0.81) – precision выше (0.85), но recall ниже (0.77), возможны ложноотрицательные.\
Вывод: Модель в целом хорошо распознает эти классы, но recall немного ниже, что говорит о возможных пропущенных объектах.\

- Классы с низкими результатами (f1-score < 0.70)\
Класс 2: (f1-score 0.68) – высокая precision (0.81), но низкий recall (0.58), модель часто ошибается в сторону "непредсказания" этого класса.\
Класс 10: (f1-score 0.52) – самый слабый класс, recall всего 0.45, означает, что модель плохо находит объекты этого класса.\
Вывод: Класс 10 - самый проблемный, модель часто не распознает его, что вызвано малым количеством обучающих данных или высокой схожестью с другими классами.

### Выводы

1. В анализе представлены результаты четырех моделей классификации:

Логистическая регрессия\
SVC (Support Vector Classifier)\
CatBoost\
Оптимизированный CatBoost с Optuna\
Оценка моделей выполнена с помощью метрик ROC AUC, precision, recall, f1-score и accuracy.

2. Сравнение моделей по метрике ROC AUC
ROC AUC отражает способность модели различать классы. Чем выше значение, тем лучше модель:

- Logistic Regression: 0.908
- SVC: 0.935
- CatBoost: 0.984
- Optuna + CatBoost: 0.982\
CatBoost и его оптимизированная версия показали лучшие результаты по метрике ROC AUC, существенно превосходя линейные модели.

3. Сравнение по метрике accuracy (доля правильных ответов)
- Logistic Regression: 0.70
- SVC: 0.75
- CatBoost: 0.88
- Optuna + CatBoost: 0.88\
CatBoost и его оптимизированная версия также показывают наивысшую точность.

4. Анализ precision, recall и f1-score по классам

Логистическая регрессия:
Хорошие результаты по классу 1 (f1-score 0.95) и 9 (f1-score 0.77).
Провал на классах 2, 3, 7, 8 (f1-score ≈ 0).
В целом, модель не справляется с дисбалансом классов.

SVC:
Значительное улучшение по большинству классов.
Например, класс 2 теперь имеет recall 0.77 (значительно лучше, чем 0 в логистической регрессии).
Улучшились результаты на классах 3, 5, 6, 7, 8, но класс 4 стал хуже (recall 0.46).
SVC лучше справляется с редкими классами, но все еще далек от оптимального.

CatBoost:
Существенный рост f1-score для всех классов.
Класс 2 (f1-score 0.65) и 10 (f1-score 0.51) показывают заметное улучшение.
Большинство классов имеют f1-score > 0.75, что говорит о сбалансированном распознавании.
Модель явно лучше улавливает сложные зависимости в данных.

Optuna + CatBoost:
Почти идентичные результаты с обычным CatBoost.
Незначительные улучшения для классов 2, 3, 7.
Однако f1-score по классу 10 слегка упал (с 0.51 до 0.52).

Оптимизация дала минимальные улучшения, но незначительно повлияла на метрики.

5. Выводы

Логистическая регрессия — худший вариант, плохо обрабатывает редкие классы.
SVC показал улучшение, но все еще плохо работает с некоторыми редкими классами.
CatBoost однозначно лучшая модель, обеспечивающая наивысшую точность и сбалансированность по всем классам.
Оптимизация CatBoost через Optuna не дала значительных улучшений, но подтвердила, что градиентный бустинг уже является оптимальным выбором.